# DATA EXPLORATION GAS SALES

In [10]:
import pandas as pd
from typing import List

In [11]:
# Define sample path
DEMO_DATA_FILE_PATH = r"C:\Users\ecastaneda\Documents\Dataton\gas-sales-prediction\data\raw\2019\Ene 2019 ATQ Y EC.csv"

In [12]:
# Read df
sample_df = pd.read_csv(DEMO_DATA_FILE_PATH, low_memory=False, thousands=',')
sample_df.head(10)

,Empresa,Region,Filial,Tipo,Punto de venta,%Key_Cliente,%Key_FechaID,%Key_Folio,%Key_Ticket,Litros,$
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2545243,23837423
1,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190103.0,3000-5262-34E4-2000-4B77,yx6E1H,15,142
2,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190103.0,3000-5262-34E8-3800-4B78,yx6E1I,22,215
3,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190103.0,3000-5262-34ED-6000-4B79,yx6E1K,22,215
4,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190103.0,3000-5262-34F6-B000-4B7A,yx6E1L,10,98
5,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190103.0,3000-5262-3502-7800-4B7B,yx6E1N,5,49
6,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190103.0,3000-5262-38DE-8000-4B9E,yx6E38,11,104
7,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190104.0,3000-5262-4556-6800-4BAD,yx6IRQ,27,264
8,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190104.0,3000-5262-45B0-4000-4BAF,yx6IRZ,12,117
9,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190105.0,3000-5262-5459-7000-4BC7,yx6LW9,12,119


In [13]:
# Show info
print(sample_df.shape)
sample_df.info()

(378157, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378157 entries, 0 to 378156
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Empresa         378156 non-null  object 
 1   Region          378156 non-null  object 
 2   Filial          378156 non-null  object 
 3   Tipo            378156 non-null  object 
 4   Punto de venta  378156 non-null  object 
 5   %Key_Cliente    378156 non-null  object 
 6   %Key_FechaID    378156 non-null  float64
 7   %Key_Folio      378156 non-null  object 
 8   %Key_Ticket     378156 non-null  object 
 9   Litros          378157 non-null  int64  
 10  $               378157 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 31.7+ MB


In [14]:
# Remove totals
def remove_totals(data_df: pd.DataFrame, column: str) -> pd.DataFrame:
    filtered_df = data_df[data_df[column].notna()]
    return filtered_df

sample_df = remove_totals(sample_df,"Filial")
sample_df.shape

(378156, 11)

In [15]:
# Convert type
sample_df.convert_dtypes()

,Empresa,Region,Filial,Tipo,Punto de venta,%Key_Cliente,%Key_FechaID,%Key_Folio,%Key_Ticket,Litros,$
1,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190103,3000-5262-34E4-2000-4B77,yx6E1H,15,142
2,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190103,3000-5262-34E8-3800-4B78,yx6E1I,22,215
3,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190103,3000-5262-34ED-6000-4B79,yx6E1K,22,215
4,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190103,3000-5262-34F6-B000-4B7A,yx6E1L,10,98
5,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190103,3000-5262-3502-7800-4B7B,yx6E1N,5,49
...,...,...,...,...,...,...,...,...,...,...,...
378152,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Sur,TAMPICO,Autotanque,ATQ. TAMP 77 (047),TC0001,20190125,3004-D263-975C-0000-2398,y95868,4,38
378153,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Sur,TAMPICO,Autotanque,ATQ. TAMP 12 (043),TC0001,20190125,3000-C263-9562-6000-1F3A,y95962,5,46
378154,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Sur,TAMPICO,Autotanque,ATQ. TAMP 12 (043),TC0001,20190125,3000-C263-9595-4000-1F3C,y95964,1,13
378155,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Sur,TAMPICO,Autotanque,ATQ. TAMP 12 (043),TC0001,20190125,3000-C263-961E-0000-1F3F,y95969,7,66


In [16]:
# Get date
def keydate_to_date(data_df: pd.DataFrame, column: str = "%Key_FechaID") -> pd.Series:
    date_series = pd.to_datetime(data_df[column].astype(str), format = "%Y%m%d")
    return date_series

sample_df["Date"] = keydate_to_date(sample_df, "%Key_FechaID")

In [18]:
sample_df["Filial"].unique()

array(['ALAMO', 'ALDAMA', 'ALLENDE', 'CD. VICTORIA', 'GALEANA', 'JIMENEZ',
       'LINARES', 'MATEHUALA', 'MONTEMORELOS', 'POZA RICA', 'REYNOSA',
       'RIO VERDE', 'SALINASVICTORIA', 'SALTILLO', 'SAN FERNANDO',
       'SAN LUIS POTOSI', 'SAN NICOLAS', 'SAN RAFAEL', 'SANTA CATARINA',
       'SOTO LA MARINA', 'TAMPICO'], dtype=object)

In [17]:
# Remove columns
COLUMNS_TO_REMOVE = ["Empresa", "%Key_Ticket","%Key_Folio", "%Key_Cliente", "%Key_FechaID"]
sample_df.drop(columns = COLUMNS_TO_REMOVE, inplace=True)
sample_df.sample(10)

KeyError: "['Empresa', '%Key_Ticket', '%Key_Folio', '%Key_Cliente', '%Key_FechaID'] not found in axis"